In [1]:
%tensorflow_version 2.x

In [2]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2020-11-24 06:43:20--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2020-11-24 06:43:20--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc10340e07e08f095fff1ae50d93.dl.dropboxusercontent.com/zip_download_get/AnIGS81aUdIHf7kSQPFvw8Qq7ZZyhTvu3e4Co5cffOQHDJRj568A7OZgwU_AVUexbpKU4YLkHZ81629BM2sI7THUVL41MjQ5RVoYviYhWqdfNA [following]
--2020-11-24 06:43:20--  https://uc10340e07e08f095fff1ae50d93.dl.dropboxusercontent.com/zip_download_get/AnIGS81aUdIHf7kSQPFvw8Qq7ZZyhTvu3e4Co5cffOQHDJRj568A7OZgwU_AVUexbpKU4YLkHZ81629BM2sI

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk import word_tokenize


In [4]:
filter=pd.read_csv(r'/content/AAAw1_JIzpuVvwteJCma0xMla?dl=0',sep=',', encoding='latin-1')

In [5]:
filter

,PK,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0.0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0.0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0.0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0.0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0.0
...,...,...,...
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0.0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0.0
1306121,ffffed09fedb5088744a,Who wins in a battle between a Wolverine and a...,0.0
1306122,PKÐÞ¾±´>g´>gPK,NaN,NaN


In [6]:
filter.isnull().sum()

PK            0
question_text    2
target           2
dtype: int64

In [7]:
for col in ['question_text','target']:
  print(col,',',filter[col].value_counts())

question_text , My son is four years old. He has medullo blastoma brain cancer, what is his life expectancy?                1
Why the washbasin of the train, in general class always dirty?                                              1
How can you make iced coffee with milk?                                                                     1
Can you help me to find and identify my PhD idea? Deep learning and Big Data Twitter Sentiment analysis?    1
Does the law of equivalent exchange apply to chemistry?                                                     1
                                                                                                           ..
How can I share a post on a Quora blog without it being duplicated on my own blog?                          1
What is the meaning of 'banking on your standards to be lower than mine'?                                   1
Whatfoodfor a singer?                                                                                   

In [8]:
filter.drop([1306122,1306123],0,inplace=True)

In [9]:
filter.isnull().sum()

PK            0
question_text    0
target           0
dtype: int64

In [10]:
filter.drop(['PK'],1,inplace=True)

In [11]:
filter.dtypes

question_text     object
target           float64
dtype: object

In [12]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model

from nltk import word_tokenize

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [13]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2020-11-24 06:44:40--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2020-11-24 06:44:40--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2020-11-24 06:44:40--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [14]:
!unzip /content/glove.42B.300d.zip 

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [15]:
!rm /content/glove.42B.300d.zip

In [16]:
embeding_index={}

f=open('glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=coefs
f.close()

In [17]:
len(embeding_index['provide'])

300

In [18]:
embeding_index['provide']

array([ 2.1902e-01, -6.6693e-02,  1.0538e-01, -2.0213e-01,  3.4797e-01,
       -8.4570e-03, -4.0350e+00,  3.4057e-01,  1.0484e-01,  2.1855e-01,
       -4.0837e-01,  2.7482e-01,  1.1953e-01,  7.4335e-02, -1.7069e-01,
       -4.0008e-01, -7.9120e-03, -3.9702e-01,  5.1159e-02, -2.3763e-02,
       -4.6144e-01,  4.7578e-02, -1.8110e-01,  3.9715e-03,  1.5718e-01,
       -1.8605e-01,  3.4680e-01,  1.7597e-01, -5.9390e-02, -3.0119e-01,
       -2.8326e-01, -2.6415e-01, -1.9358e-02,  6.7555e-03,  3.3796e-01,
       -3.5508e-01,  1.0336e-02, -5.0465e-01,  1.3932e-01, -2.7452e-01,
       -1.6377e-01,  5.0877e-01, -1.4303e-01, -2.0842e-01,  2.0463e-01,
        7.3660e-02, -1.8813e-01,  2.8356e-01, -2.7095e-01, -2.5136e-01,
       -1.2715e-01,  2.1164e-01, -4.4701e-01,  9.5515e-02,  4.5539e-01,
       -6.0036e-02,  1.4488e-02, -2.9433e-01, -5.5977e-01,  2.8814e-01,
        9.5750e-03,  2.2739e-01, -8.2802e-02,  3.0510e-01,  3.8501e-02,
        3.2902e-01, -1.1190e-02, -1.2909e-01,  3.6758e-01, -2.29

In [19]:
len(embeding_index)

1917494

In [20]:
X=filter['question_text']
Y=filter['target']

In [21]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1044897,), (261225,), (1044897,), (261225,))

In [22]:
class_weight=class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
print(class_weight)

[0.53290571 8.0974659 ]


In [23]:
class_weight_dict = dict(enumerate(class_weight))
class_weight_dict

{0: 0.5329057087222568, 1: 8.097465902045878}

In [24]:
x_train

395145             What year did cars start having internet?
453957     Why is there a law in America that allows a wo...
528957     What are some "unlucky" things that are consid...
742642     What do accountants do and do they get paid well?
936751     Do you think that Mike Pence is not a prostitu...
                                 ...                        
509237          Clock wise rotation is positive or negative?
1137731    What is your best customer service experience ...
351177     The man who raped me when I was 14 is on the S...
640214                   How do I build muscle fast at home?
918435     Do girls always have lube coming out their vag...
Name: question_text, Length: 1044897, dtype: object

In [25]:
from nltk import word_tokenize

In [26]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [27]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

412


In [28]:
np.percentile(sent_lens,99)

44.0

In [29]:
max_len=44

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [30]:
tk.word_index

{'the': 1,
 'what': 2,
 'is': 3,
 'a': 4,
 'to': 5,
 'in': 6,
 'of': 7,
 'i': 8,
 'how': 9,
 'and': 10,
 'do': 11,
 'are': 12,
 'for': 13,
 'you': 14,
 'can': 15,
 'why': 16,
 'it': 17,
 'my': 18,
 'that': 19,
 'if': 20,
 'with': 21,
 'on': 22,
 'or': 23,
 'have': 24,
 'be': 25,
 'does': 26,
 'from': 27,
 'your': 28,
 'an': 29,
 'which': 30,
 'should': 31,
 'when': 32,
 'get': 33,
 'best': 34,
 'would': 35,
 'as': 36,
 'people': 37,
 'some': 38,
 'there': 39,
 'who': 40,
 'will': 41,
 'like': 42,
 'at': 43,
 'not': 44,
 'about': 45,
 'by': 46,
 'they': 47,
 'did': 48,
 'was': 49,
 'any': 50,
 'we': 51,
 'so': 52,
 'good': 53,
 'me': 54,
 'their': 55,
 'one': 56,
 'india': 57,
 'has': 58,
 'after': 59,
 'most': 60,
 'where': 61,
 'make': 62,
 'this': 63,
 'but': 64,
 'more': 65,
 'all': 66,
 'think': 67,
 'many': 68,
 'between': 69,
 'time': 70,
 'than': 71,
 'much': 72,
 'other': 73,
 'life': 74,
 'someone': 75,
 'use': 76,
 'he': 77,
 'way': 78,
 'out': 79,
 'am': 80,
 'know': 81,
 'u

In [31]:
vocab_size

196188

In [33]:
seq_train_matrix.shape

(1044897, 44)

In [34]:
seq_train

[[2, 103, 48, 1024, 121, 176, 537],
 [16,
  3,
  39,
  4,
  337,
  6,
  261,
  19,
  3175,
  4,
  321,
  5,
  1329,
  4,
  223,
  24,
  4,
  329,
  10,
  3176,
  172,
  13,
  329,
  382,
  146,
  20,
  77,
  3,
  4,
  2443],
 [2, 12, 38, 15061, 147, 19, 12, 313, 5685, 6, 73, 2496],
 [2, 11, 10408, 11, 10, 11, 47, 33, 789, 309],
 [11, 14, 67, 19, 4463, 6342, 3, 44, 4, 6571, 36, 459, 123],
 [9, 11, 8, 174, 350, 27, 5389, 1201],
 [16,
  3,
  17,
  19,
  8,
  111,
  42,
  1143,
  234,
  10,
  44,
  356,
  931,
  146,
  569,
  56,
  811,
  4652,
  18,
  1288,
  36,
  1750,
  356],
 [16, 11, 37, 67, 66, 544, 535, 2530, 12, 848, 2152, 1822, 22632],
 [3,
  1,
  1766,
  28,
  4813,
  3,
  46677,
  324,
  105,
  695,
  2,
  1185,
  8990,
  8188,
  129,
  1872,
  29,
  2109,
  202,
  63,
  35623,
  19,
  1334,
  41,
  3532,
  50,
  1185,
  7,
  322,
  161,
  47,
  773,
  81,
  2,
  35,
  25,
  457],
 [2, 3, 1, 688, 7, 9943, 6, 9272],
 [2, 12, 1, 130, 565, 7, 11582, 10, 2, 3, 55, 510],
 [9,
  11,


In [35]:
x_train.shape

(1044897,)

In [36]:
len(seq_train),len(seq_test)

(1044897, 261225)

In [37]:
seq_train_matrix.shape, seq_test_matrix.shape

((1044897, 44), (261225, 44))

In [38]:
embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

In [39]:
 len(embedding_matrix)

196189

In [40]:
embedding_matrix.shape

(196189, 300)

In [41]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.20838   , -0.14932001, -0.017528  , ..., -0.54066002,
         0.21199   , -0.0094357 ],
       [ 0.13877   , -0.0021201 , -0.091039  , ...,  0.024614  ,
         0.46514001,  0.11363   ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [42]:
inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [44]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      [(None, 44)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 44, 300)           58856700  
_________________________________________________________________
lstm (LSTM)                  (None, 512)               1665024   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)              

In [50]:
filepath='/content/weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True)

In [51]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [52]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [53]:
model.fit(seq_train_matrix,y_train,
              epochs=10,
              class_weight={0: 0.5331085383324966, 1: 8.050922287457816},
              batch_size=1000,
              validation_data=(seq_test_matrix,y_test),
              callbacks = [earlystop, checkpoint])

Epoch 1/10
1045/1045 [==============================] - ETA: 0s - loss: 0.2986 - accuracy: 0.8718
Epoch 00001: val_loss improved from inf to 0.29930, saving model to /content/weights-01-0.2993.h5
1045/1045 [==============================] - 280s 268ms/step - loss: 0.2986 - accuracy: 0.8718 - val_loss: 0.2993 - val_accuracy: 0.8785
Epoch 2/10
1045/1045 [==============================] - ETA: 0s - loss: 0.2505 - accuracy: 0.8948
Epoch 00002: val_loss improved from 0.29930 to 0.25639, saving model to /content/weights-02-0.2564.h5
1045/1045 [==============================] - 279s 267ms/step - loss: 0.2505 - accuracy: 0.8948 - val_loss: 0.2564 - val_accuracy: 0.8966
Epoch 3/10
1045/1045 [==============================] - ETA: 0s - loss: 0.2280 - accuracy: 0.9015
Epoch 00003: val_loss improved from 0.25639 to 0.25243, saving model to /content/weights-03-0.2524.h5
1045/1045 [==============================] - 280s 268ms/step - loss: 0.2280 - accuracy: 0.9015 - val_loss: 0.2524 - val_accuracy: 

In [57]:
best_model = load_model('/content/weights-09-0.2005.h5')

In [ ]:
best_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      [(None, 44)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 44, 300)           58837200  
_________________________________________________________________
lstm (LSTM)                  (None, 512)               1665024   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)              

In [58]:
p = best_model.predict(seq_test_matrix)
roc_auc_score(y_test,p)

0.9630063260917258

In [59]:
best_model.save('./SFM_spam_filter_model_quora_questions.h5')